In [0]:
from pyspark.sql.functions import input_file_name

catalog_name = "users"
schema_name = "yukiteru_koide"
base_path = "/Volumes/users/yukiteru_koide/yukiterumart_etl_newdata"

# CatalogとSchemaを選択
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")


In [0]:
from pyspark.sql.functions import col

# Auto LoaderでCSVファイルを読み込み
bronze_stream = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("cloudFiles.schemaLocation", f"{base_path}/_schema/bronze")  # schema log保存先を指定
    .load(base_path)
    .withColumn("source_file", col("_metadata.file_path"))
)

# BronzeテーブルにDeltaとして書き出し
bronze_stream.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", f"{base_path}/_checkpoints/bronze") \
    .trigger(once=True) \
    .table(f"{catalog_name}.{schema_name}.bronze_transactions")